# Quick start

- To check data and model if it preditcs or not.
- This is only for intent dataset
- Model implemented in for 8640 data samples after filtering

# 1)- Import Key Modules

In [1]:
# support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [3]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [4]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


# 2)- Loading Data

In [5]:
data=pd.read_pickle('file_clean_intent.pkl')

In [6]:
data.dep.nunique()

30

In [8]:
counts=data['dep'].value_counts()
df = data.loc[data['dep'].isin(counts.index[counts > 150])]

In [9]:
print(data.shape)
print(df.shape)

(9477, 4)
(8640, 4)


In [11]:
df.dep.nunique()

10

In [12]:
myfeat=["Order management","product complaints - products (Reklamation Produkte)","Software/Webshop/App",
        "Shipping issues","Customer feedback","ShareWithSaal","Payment (Bezahlung)",
        "product complaints - colours (Reklamation Farben)","Product (Produkt)","Marketing"]

### Data Clean

In [13]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['clean'] = df['firstmessage'].apply(clean_text)

### Train-Test

In [14]:
train_size = int(len(df) * 0.70)
train_posts = df['clean'][:train_size]
train_tags = df['dep'][:train_size]

test_posts = df['clean'][train_size:]
test_tags = df['dep'][train_size:]

In [15]:
print(train_posts.shape)

(6048,)


### Tokenization

In [16]:
max_words = 10000
tokenize = text.Tokenizer(num_words=max_words,char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [17]:
print(x_train.shape)

(6048, 10000)


In [18]:
x_train[:5]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### encoder

In [19]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags) 
y_test = encoder.transform(test_tags)

In [20]:
num_classes = np.max(y_train) + 1 #strings start from 0 index. To count them from 1-10
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [21]:
print(y_train.shape)
print(y_test.shape)

(6048, 10)
(2592, 10)


### Model Building

In [54]:
batch_size = 32
epochs = 10
shape_of_input=max_words # 10000 words
EMBEDDING_DIM=10000
MAX_NB_WORDS = 5000# The maximum number of words to be used. (most frequent)

In [67]:
model = keras.Sequential()
model.add(keras.layers.Embedding(10000, 512))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dropout(.2))#The dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle
model.add(keras.layers.Dense(512,kernel_initializer='uniform', input_shape=(shape_of_input,), activation="relu"))
model.add(keras.layers.Dropout(.2))
model.add(keras.layers.Dense(num_classes, activation="softmax"))

In [68]:
"""

from keras.layers import Dense, Embedding
model = Sequential()
model.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM, input_length=max_words))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))#The dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle
#model.add(Dense(num_classes)) # there are 10 classes i.e at output layer
#model.add(Activation('softmax'))
model.add(Dense(10, activation='softmax'))

"""

'\n\nfrom keras.layers import Dense, Embedding\nmodel = Sequential()\nmodel.add(Embedding(MAX_NB_WORDS,EMBEDDING_DIM, input_length=max_words))\nmodel.add(Dense(512, activation="relu"))\nmodel.add(Dropout(0.2))#The dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle\n#model.add(Dense(num_classes)) # there are 10 classes i.e at output layer\n#model.add(Activation(\'softmax\'))\nmodel.add(Dense(10, activation=\'softmax\'))\n\n'

In [69]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [70]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 512)         5120000   
_________________________________________________________________
global_average_pooling1d_6 ( (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 5,387,786
Trainable params: 5,387,786
Non-trainable params: 0
____________________________________________

In [71]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 5443 samples, validate on 605 samples
Epoch 1/10
5443/5443 [==============================] - 158s 29ms/sample - loss: 1.7031 - accuracy: 0.5054 - val_loss: 1.5605 - val_accuracy: 0.6083
Epoch 2/10
5443/5443 [==============================] - 184s 34ms/sample - loss: 1.6890 - accuracy: 0.5074 - val_loss: 1.5431 - val_accuracy: 0.6083
Epoch 3/10
5443/5443 [==============================] - 191s 35ms/sample - loss: 1.6869 - accuracy: 0.5074 - val_loss: 1.5261 - val_accuracy: 0.6083
Epoch 4/10
5443/5443 [==============================] - 156s 29ms/sample - loss: 1.6892 - accuracy: 0.5074 - val_loss: 1.5539 - val_accuracy: 0.6083
Epoch 5/10
5443/5443 [==============================] - 157s 29ms/sample - loss: 1.6860 - accuracy: 0.5074 - val_loss: 1.5607 - val_accuracy: 0.6083
Epoch 6/10
5443/5443 [==============================] - 158s 29ms/sample - loss: 1.6843 - accuracy: 0.5074 - val_loss: 1.5473 - val_accuracy: 0.6083
Epoch 7/10
5443/5443 [==============================] - 159

One thing has improved from previous model, there is no overfitting pattern.

In [72]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

2592/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

Test accuracy: 0.63117284


# END OF NOTEBOOK